In [2]:
import numpy as np
from PIL import Image
import os

In [3]:
originals_dir = "./originals/"
spacials_dir = "./spacials/"
freqs_dir = "./freqs/"

In [4]:
images = []
square_side = 360
square_dimensions = np.array([square_side, square_side])

for idx, filename in enumerate(os.listdir(originals_dir)):
    if not filename.endswith(".jpg"):
        continue

    # Pull image
    original_path = os.path.join(originals_dir, filename)
    image = Image.open(original_path)
    dimensions = np.array(np.array(image).shape[:2])

    # Rescale image
    min_dimension = min(dimensions)
    rescale_factor = square_side / min_dimension
    scaled_dimensions = np.flip((np.array(dimensions) * rescale_factor).astype('int16'))
    rescaled = image.resize(scaled_dimensions)

    # Crop image
    offset = ((scaled_dimensions - square_dimensions) / 2).astype('int16')
    cropped_dimensions = np.concatenate([offset, square_dimensions + offset])
    cropped = rescaled.crop(cropped_dimensions)

    # Save transformed image
    spacial_path = os.path.join(spacials_dir, f"{idx}.jpg")
    cropped.save(spacial_path)

    # Move to frequency domain
    cropped_grey = np.array(cropped.convert('L'))
    freq_array = np.fft.fftshift(np.fft.fft2(cropped_grey))
    freq_array = np.log(abs(freq_array))
    freq_array = np.floor(freq_array * 255 / freq_array.max()).astype(np.uint8)
    freq_image = Image.fromarray(freq_array, 'L')

    # Save FFT image
    freq_path = os.path.join(freqs_dir, f"{idx}.jpg")
    freq_image.save(freq_path)